In [1]:
import pysam
import numpy as np
import cPickle as pickle

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from pyUMI import umi_stats as stats
from pyUMI import multimap_solver as solver

In [3]:
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
from pyUMI import repeat_solver

In [5]:
#AAAACT  AAAGTT  ATATAG  ATCAAA  CCCTGG  CCGGAC  GACCGC  GGCGTC  GTTCGA  TAAAGT  TCTGCA  TTAATC

In [6]:
st = pysam.AlignmentFile("/data/UMI/data/MUS/sample_AAAACT.bam","rb")
#fst = repeat_solver.filter_alignmentFile(st, cell_barcode="TTAATC",repeats=True)

-----------------

In [37]:
st = pysam.AlignmentFile("/data/UMI/data/MUS/sample_AAAACT.bam","rb")

In [7]:
q = [k for k in stats.multimap_generator(st)]

In [20]:
g = [k.get_tag("GE") for k in q if k.has_tag("GE") and not "ENSMUS" in k.get_tag("GE") ]

In [21]:
from collections import Counter
gg = Counter(g)
len(gg)
#gg.most_common(10)

128363

In [22]:
gg

Counter({'B2_Mm2_dup42359': 8,
         'B2_Mm2_dup36715': 1,
         'L1Md_T_dup12440': 2,
         'B2_Mm2_dup36713': 9,
         'B1_Mus2_dup57778': 9,
         'B2_Mm2_dup36711': 2,
         'B2_Mm2_dup36710': 1,
         'L1Md_T_dup21944': 1,
         'B1_Mus2_dup57776': 1,
         'L1Md_T_dup21940': 3,
         'B2_Mm2_dup36719': 29,
         'RLTR31D_MM_dup384': 2,
         'L1Md_F2_dup5498': 3,
         'B1_Mus1_dup5959': 2,
         'L1Md_T_dup15798': 2,
         'L1Md_T_dup15797': 13,
         'L1Md_T_dup15793': 338,
         'L1Md_T_dup21949': 3,
         'B2_Mm2_dup2111': 12,
         'B2_Mm2_dup2112': 4,
         'B2_Mm2_dup31846': 9,
         'B1_Mus2_dup13587': 10,
         'B2_Mm2_dup31844': 1,
         'B2_Mm2_dup2116': 2,
         'B2_Mm2_dup2119': 19,
         'B2_Mm1t_dup14081': 6,
         'B2_Mm2_dup31849': 20,
         'L1_Mus3_dup12378': 9,
         'B2_Mm1a_dup6949': 3,
         'B2_Mm1a_dup6948': 5,
         'RSINE1_dup3457': 2,
         'B2_Mm1a_dup6945': 3

In [125]:
%%time
st.reset()

mapped = st.mapped
unmapped = st.unmapped
multimaps, uniques = stats.count_um(st)

CPU times: user 28.5 s, sys: 191 ms, total: 28.7 s
Wall time: 28.7 s


In [126]:
print mapped
print unmapped
print multimaps
print uniques

17752333
664548
14353994
3398339


In [48]:
mm_hashtable = solver.build_multimapping_hashtable(st)

In [38]:
uniqs_hashtable, uniqs_map = solver.build_uniques_associations(st, cell_barcodes=["AAAACT"])

In [110]:
umi = solver.query_uniques_hashtable(uniqs_hashtable, keyword='umi')

In [94]:
mm_genes = solver.query_multimap_hashtable(mm_hashtable, keyword='gene')

In [103]:
qn = solver.query_multimap_hashtable(mm_hashtable, keyword='umi')

In [39]:
uniqs_genes = solver.get_uniquesmap_genes(uniqs_map)

In [88]:
len(uniqs_genes)

2726958

In [86]:
uniqs_regions = solver.get_uniquesmap_genes(uniqs_map, gene=False)

In [89]:
len(uniqs_regions)

3398339

In [42]:
uniqs_repeats = [a for a in uniqs_genes if not("ENS" in a)]

In [92]:
len(solver.get_bamfile_genes(st))

12589260

In [46]:
len(genes)

12589260

In [44]:
len(uniqs_repeats)

194052

In [92]:
m1 = solver.build_multimapping_hashtable(st1)
uh1,u1 = solver.build_uniques_associations(st1, cell_barcodes=["AAAACT"])

In [16]:
len(u1)

3398339

In [7]:
fst1 = repeat_solver.filter_alignmentFile(st1, cell_barcode="U",repeats=True)
fst1 = pysam.AlignmentFile("/data/UMI/data/MUS/sample_U_filtered.bam","rb")

In [12]:
fst1.reset()
reads =  fst1.fetch(until_eof=True)

In [13]:
tags =[]
for read in reads:
    tags.append(read.get_tag("GE"))

In [15]:
uh1_r,u1_r = solver.build_uniques_associations(fst1, cell_barcodes=["AAAACT"])

In [25]:
ur_tags = [u1_r[k][0] for k in u1_r]

In [ ]:
solver.get_uniquesmap_genes(uniques)

In [27]:
len(ur_tags)

194052

In [37]:
from collections import Counter
urt = Counter(ur_tags)
len(urt)
urt_list = [k for k in urt]

In [38]:
len(urt_list)

31416

In [40]:
ranking = []
st1.reset()
reads = st1.fetch(until_eof=True)

for read in reads:
    if (read.has_tag("GE")):
        ranking.append(read.get_tag("GE"))

In [56]:
ranking_dict = Counter(ranking)

In [59]:
import operator

ranking_dict_sorted = sorted(ranking_dict.items(), key=operator.itemgetter(1),reverse=True)


In [61]:
#ranking_dict_sorted
ranked_repeats = [(i,j) for (i,j) in ranking_dict_sorted if not("ENSMUS" in i)]

In [69]:
ranked_unique_repeats = sorted(urt.items(), key=operator.itemgetter(1),reverse=True)

    Gene                      Uniquely mapped reads
    'B2_Mm1a_dup12765'        1484
    'B2_Mm1a_dup17081'        1309
    'B2_Mm1t_dup4289'         1124
    'L1_Mur2_dup15158'        1015
    'PB1D7_dup8929'           939

In [86]:
with open('/data/UMI/data/MUS/AAAACT_ranked_unique_repeats.pkl', 'wb') as handle:
    pickle.dump(ranked_unique_repeats, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [72]:
unique_repeats = [k for k in urt]

In [75]:
top_counter = 5
for item in ranked_repeats:
    if item[0] in unique_repeats:
        print item
        top_counter -= 1
    if not(top_counter):
        break

('L1_Mus2_dup2959', 15829)
('L1_Mus1_dup17730', 11470)
('B2_Mm1a_dup7097', 8880)
('B2_Mm1t_dup14771', 7747)
('B2_Mm1a_dup7601', 7650)


    Gene                  Frequency    Uniques

    'L1_Mus2_dup2959',    15829        5
    'L1_Mus1_dup17730'    11470        76
    'B2_Mm1a_dup7097'     8880         1
    'B2_Mm1t_dup14771'    7747         1
    'B2_Mm1a_dup7601'     7650         3

-----------------------------------------

    ERVB4_2-LTR_MM_dup63

    ERVB4_2-LTR_MM_dup63    chrY    +    29718275    29718800

    Read length = 50bp

    Read name = HISEQ:280:C9J9KANXX:2:2115:15792:76261

    ('ERVB4_2-LTR_MM_dup63', 'chrY', 29718465)

    Location = chrY:29,718,472

    Read name = HISEQ:280:C9J9KANXX:1:1310:7181:38503

    ('ERVB4_2-LTR_MM_dup63', 'chrY', 29718496)

    Location = chrY:29,718,546

    275 800

    472 522

    546 596

